In [2]:
import os

import autogen

config_list = autogen.config_list_from_json(env_or_file="../OAI_CONFIG_LIST")

# Put the OpenAI API key into the environment
os.environ["OPENAI_API_KEY"] = config_list[0]["api_key"]

In [ ]:
from graphrag_sdk import (
    Attribute,
    AttributeType,
    Entity,
    Ontology,
    Relation,
)

# Attraction + Restaurant + City + Country Ontology
trip_data_ontology = Ontology()


# Manually created Ontology by adding entities and relations
trip_data_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
trip_data_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
trip_data_ontology.add_entity(
    Entity(
        label="Restaurant",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="rating",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
            Attribute(
                name="food_type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
trip_data_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
trip_data_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Restaurant",
        target="City",
    )
)

trip_data_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
trip_data_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
trip_data_ontology.add_entity(
    Entity(
        label="Attraction",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
trip_data_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
trip_data_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Attraction",
        target="City",
    )
)

In [ ]:
from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.graph_rag.document import Document, DocumentType
from autogen.agentchat.contrib.graph_rag.falkor_graph_query_engine import FalkorGraphQueryEngine
from autogen.agentchat.contrib.graph_rag.falkor_graph_rag_capability import FalkorGraphRagCapability

# Auto generate graph schema from unstructured data
input_paths = [
    "../test/agentchat/contrib/graph_rag/trip_planner_data/attractions.json",
    "../test/agentchat/contrib/graph_rag/trip_planner_data/cities.json",
    "../test/agentchat/contrib/graph_rag/trip_planner_data/restaurants.json",
]
input_documents = [Document(doctype=DocumentType.TEXT, path_or_url=input_path) for input_path in input_paths]

# Create FalkorGraphQueryEngine
query_engine = FalkorGraphQueryEngine(
    name="trip_data",
    host="172.18.0.3",  # Change
    port=6379,  # if needed
    ontology=trip_data_ontology,
)

# Ingest data and initialize the database
query_engine.init_db(input_doc=input_documents)

# Create a ConversableAgent (no LLM configuration)
graph_rag_agent = ConversableAgent(
    name="graph_rag_agent",
    human_input_mode="NEVER",
)

# Associate the capability with the agent
graph_rag_capability = FalkorGraphRagCapability(query_engine)
graph_rag_capability.add_to_agent(graph_rag_agent)

# Create a user proxy agent to converse with our RAG agent
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
)

user_proxy (to graph_rag_agent):

Find 3 restaurant in Milan.

--------------------------------------------------------------------------------
graph_rag_agent (to user_proxy):

Bros' Burger in Milan is known for its gourmet burgers and offers a relaxed vibe.

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Find 3 restaurant in Milan.', 'role': 'assistant', 'name': 'user_proxy'}, {'content': "Bros' Burger in Milan is known for its gourmet burgers and offers a relaxed vibe.", 'role': 'user', 'name': 'graph_rag_agent'}], summary="Bros' Burger in Milan is known for its gourmet burgers and offers a relaxed vibe.", cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=['exit'])

In [12]:
user_proxy.initiate_chat(graph_rag_agent, message="Find 3 restaurants in Rome.")

user_proxy (to graph_rag_agent):

Find 3 restaurants in Rome.

--------------------------------------------------------------------------------
graph_rag_agent (to user_proxy):

Three restaurants in Rome are Trattoria da Enzo, known for its traditional Roman dishes and welcoming atmosphere; Il Pagliaccio, an elegant Michelin-starred restaurant offering contemporary Italian cuisine; and Tonnarello, a casual spot for pasta, meatballs, and other simple Roman dishes with patio seating and acoustic guitar music.

--------------------------------------------------------------------------------
user_proxy (to graph_rag_agent):

Find 3 attractions in Rome.

--------------------------------------------------------------------------------
graph_rag_agent (to user_proxy):

Three attractions in Rome are the Colosseum, an ancient amphitheater known for gladiatorial contests and public spectacles; the Vatican Museums, a complex of museums and galleries showcasing works of art collected by Popes over

ChatResult(chat_id=None, chat_history=[{'content': 'Find 3 restaurants in Rome.', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'Three restaurants in Rome are Trattoria da Enzo, known for its traditional Roman dishes and welcoming atmosphere; Il Pagliaccio, an elegant Michelin-starred restaurant offering contemporary Italian cuisine; and Tonnarello, a casual spot for pasta, meatballs, and other simple Roman dishes with patio seating and acoustic guitar music.', 'role': 'user', 'name': 'graph_rag_agent'}, {'content': 'Find 3 attractions in Rome.', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'Three attractions in Rome are the Colosseum, an ancient amphitheater known for gladiatorial contests and public spectacles; the Vatican Museums, a complex of museums and galleries showcasing works of art collected by Popes over centuries; and the Trevi Fountain, a Baroque fountain famous for its stunning sculptures and the tradition of tossing coins.', 'role': 'user', 'name': '